In [49]:
import pandas as pd
from utils import WACC

# Refining

In [50]:
CONV_MJ_TO_MMBTU = 1/1055.05585 #MMBTU/MJ
EFF_H2_SMR = 159.6 #MJ/kgH2

In [51]:
lcoh_res_df = pd.read_excel('./input_data/lcoh_lcoe_res.xlsx')
res_be_df = pd.DataFrame(columns=['RES', 'Breakeven price ($/MMBtu)'])
res_be_df['RES'] = lcoh_res_df['RES']
res_be_df['Breakeven price ($/MMBtu)'] = lcoh_res_df.apply(lambda x:(x['LCOH']-x['H2 PTC'])/(EFF_H2_SMR*CONV_MJ_TO_MMBTU), axis=1)
res_be_df

,RES,Breakeven price ($/MMBtu)
0,Grid PEM,34.837997
1,Wind,24.062677
2,Solar PV-E,31.069940
3,SMR 89% CCUS,7.800538


In [52]:
res_be_df.to_csv('./results/res_be_refining.csv', header=True, index=False)


# Ammonia

In [53]:
auxNucNH3CAPEX = 64511550 #$/year
auxNucNH3LT = 20 # years

ngNH3NGConsRate = 30.82# MMBtu/tNH3
ngNH3ElecCons = 0.061 # MWh/tNH3

In [54]:
lco_df = pd.read_excel('./input_data/lcoh_lcoe_res.xlsx', index_col=0)
res_list = list(lco_df.index)

In [55]:
ammonia_df = pd.read_excel('./h2_demand_ammonia_us_2022.xlsx', sheet_name='processed')
nh3_plant_ids = list(ammonia_df['id'])

In [56]:
res_be_df= pd.DataFrame(columns=['id', 'Grid PEM', 'Wind','Solar PV-E', 'SMR 89% CCUS'])
res_be_df['id'] = ammonia_df['id']
res_be_df.set_index('id', inplace=True)

In [57]:
from opt_deployment_ammonia import get_ammonia_plant_demand
def compute_res_be_ammonia(lcoh, h2ptc, lcoe, plant_id):
  ammonia_capacity, h2_dem_kg_per_day, elec_dem_MWe, state, la, lo = get_ammonia_plant_demand(plant_id)#kg/year
  # NG equivalent demand
  ng_dem = ammonia_capacity*ngNH3NGConsRate #MMBtu/year
  # CAPEX costs for conversion to renewable energy
  crf = WACC / (1 - (1/(1+WACC)**auxNucNH3LT) ) 
  costs_conversion =auxNucNH3CAPEX*crf
  # Electricity costs
  costs_electricity_res = elec_dem_MWe*365*24*lcoe
  costs_electricity_ng = ammonia_capacity*ngNH3ElecCons*lcoe
  # Hydrogen costs
  costs_h2 = h2_dem_kg_per_day*365*(lcoh-h2ptc)
  # BE
  res_be = (costs_h2+costs_electricity_res+costs_conversion - costs_electricity_ng)/ng_dem
  return res_be

In [58]:
for plant_id in nh3_plant_ids:
  for res in res_list:
    lcoh = lco_df.loc[res, 'LCOH']
    h2ptc = lco_df.loc[res, 'H2 PTC']
    lcoe = lco_df.loc[res, 'LCOE']
    res_be = compute_res_be_ammonia(lcoh, h2ptc, lcoe, plant_id)
    res_be_df.loc[plant_id, res] = res_be
res_be_df

,Grid PEM,Wind,Solar PV-E,SMR 89% CCUS
id,,,,
CF_Do,86.430461,59.742072,77.089367,19.485018
CF_Po,86.558753,59.870364,77.217658,19.61331
CF_Ve,86.566992,59.878603,77.225898,19.62155
Ko_En,86.582862,59.894473,77.241768,19.63742
Io_We,86.605307,59.916919,77.264213,19.659865
Dy_Wa,86.638057,59.949668,77.296963,19.692615
Nu_Au,86.649978,59.96159,77.308884,19.704536
Nu_Li,86.665012,59.976624,77.323918,19.71957
Ya_Fr,86.665012,59.976624,77.323918,19.71957


In [59]:
res_be_df = res_be_df.mean().to_frame(name='Breakeven price ($/MMBtu)')
res_be_df.reset_index(inplace=True)
res_be_df.rename(columns={'index':'RES'},inplace=True )
res_be_df.to_csv('./results/res_be_ammonia.csv', header=True, index=False)

# Steel

In [60]:
lco_df = pd.read_excel('./input_data/lcoh_lcoe_res.xlsx', index_col=0)
res_list = list(lco_df.index)

In [61]:
steel_df = pd.read_excel('./h2_demand_bfbof_steel_us_2022.xlsx', sheet_name='processed')
steel_ids = list(steel_df['Plant'])
res_be_df= pd.DataFrame(columns=['id', 'Grid PEM','Wind', 'Solar PV-E', 'SMR 89% CCUS'])
res_be_df['id'] = steel_df['Plant']
res_be_df.set_index('id', inplace=True)

In [62]:
import utils

dri_co2_intensity = 40 # kgCO2/ton-DRI
shaft_CAPEX = 250 # $/tDRI/year
eaf_CAPEX = 160 # $/tsteel/year
eaf_OM  = 24.89 # $/tsteel (EAF and casting)
iron_ore_cost = 100 #$/t_ironore # $/tironore
ratio_ironore_DRI = 1.391 # tironore/tDRI
bfbof_iron_cons = 1.226 #t_ironore/t_steel
om_bfbof = 178.12 #$/t_steel

from opt_deployment_steel import get_steel_plant_demand
def compute_res_be_steel(lcoh, h2ptc, lcoe, id):
  steel_capacity_ton_per_year, h2_dem_kg_per_day, elec_dem_MWe = get_steel_plant_demand(id)#kg/year
  # Coal equivalent demand
  coal_dem = steel_capacity_ton_per_year*utils.coal_to_steel_ratio_bau*utils.coal_heat_content #MMBtu/year
  # CAPEX costs for conversion to renewable energy
  crf = WACC / (1 - (1/(1+WACC)**20) ) 
  costs_conversion= steel_capacity_ton_per_year*(eaf_CAPEX*crf*(1-utils.ITC_H2) \
                                                 + shaft_CAPEX*(1-utils.ITC_H2)*crf/utils.steel_to_dri_ratio)
  # Electricity costs
  costs_electricity_res = elec_dem_MWe*365*24*lcoe
  # Hydrogen costs
  costs_h2 = h2_dem_kg_per_day*365*(lcoh-h2ptc)
  # O&M costs for EAF
  costs_om_eaf = eaf_OM*steel_capacity_ton_per_year
  # O&M costs for BAU case
  bau_om_costs = iron_ore_cost*bfbof_iron_cons*steel_capacity_ton_per_year +\
     om_bfbof*steel_capacity_ton_per_year
  # BE
  res_be = (costs_h2+costs_electricity_res+costs_conversion + costs_om_eaf - bau_om_costs)/coal_dem
  return res_be


In [63]:
for plant_id in steel_ids:
  for res in res_list:
    lcoh = lco_df.loc[res, 'LCOH']
    h2ptc = lco_df.loc[res, 'H2 PTC']
    lcoe = lco_df.loc[res, 'LCOE']
    res_be = compute_res_be_steel(lcoh, h2ptc, lcoe, plant_id)
    res_be_df.loc[plant_id, res] = res_be
res_be_df

,Grid PEM,Wind,Solar PV-E,SMR 89% CCUS
id,,,,
U.S. Steel Granite City Works,8.155361,2.114371,5.911025,-6.668895
Cleveland-Cliffs Riverdale steel plant,8.155361,2.114371,5.911025,-6.668895
Cleveland-Cliffs Burns Harbor steel plant,8.155361,2.114371,5.911025,-6.668895
Cleveland-Cliffs Indiana Harbor steel plant,8.155361,2.114371,5.911025,-6.668895
U.S. Steel Gary Works,8.155361,2.114371,5.911025,-6.668895
Cleveland-Cliffs Dearborn steel plant,8.155361,2.114371,5.911025,-6.668895
Cleveland-Cliffs Cleveland steel plant,8.155361,2.114371,5.911025,-6.668895
Cleveland-Cliffs Middletown steel plant,8.155361,2.114371,5.911025,-6.668895
U.S. Steel Edgar Thomson Plant,8.155361,2.114371,5.911025,-6.668895


In [64]:
res_be_df = res_be_df.mean().to_frame(name='Breakeven price ($/MMBtu)')
res_be_df.reset_index(inplace=True)
res_be_df.rename(columns={'index':'RES'},inplace=True )
res_be_df.to_csv('./results/res_be_steel.csv', header=True, index=False)